In [26]:
def readInput16(infile):
    with open(infile) as f:
        return [ l.strip("\n") for l in f.readlines() ]

In [27]:
from queue import Queue
from collections import defaultdict

# directions (dx,dy)
dirs = {"N": (0,-1), "W": (-1,0), "S": (0,+1), "E": (+1,0)}

def propagate_beam(grid,start=((-1,0),"E"),verbose=False):

    q = Queue()
    energized = defaultdict(set)
    q.put(start)

    while not q.empty():
            
        # get current light beam from queue and try to move it to next position
        (x,y),m = q.get()
        
        xnext, ynext = x+dirs[m][0], y+dirs[m][1]
        pnext = (xnext,ynext)
        
        # next position is outside the grid, discard light beam
        if xnext<0 or xnext>=len(grid[0]) or ynext<0 or ynext>=len(grid):
            if verbose: print("out of grid")
            continue
        
        if verbose: print(pnext,grid[ynext][xnext],m)

        # discard if a beam already passed in this position with this direction to avoid light loops
        if m in energized[pnext]:
            continue
            
        energized[pnext].add(m)
        
        if grid[ynext][xnext]==".": # continue in same direction
            q.put((pnext,m)) 

        elif grid[ynext][xnext]=="|":
            if m=="N" or m=="S": # if coming from N or S continue in same direction
                q.put((pnext,m)) 
            elif m=="E" or m=="W": # if coming from E or W, split N and S
                q.put((pnext,"N"))
                q.put((pnext,"S"))

        elif grid[ynext][xnext]=="-":
            if m=="W" or m=="E": # if coming from W or E continue in same direction
                q.put((pnext,m)) 
            elif m=="N" or m=="S": # if coming from N or S, split W and E
                q.put((pnext,"W"))
                q.put((pnext,"E"))           

        elif grid[ynext][xnext]=='/':
            if m=="E": # -> N
                q.put((pnext,"N"))
            if m=="S": # -> W
                q.put((pnext,"W"))
            if m=="W": # -> S
                q.put((pnext,"S"))
            if m=="N": # -> E
                q.put((pnext,"E"))  

        elif grid[ynext][xnext]=='\\':
            if m=="E": # -> S
                q.put((pnext,"S"))
            if m=="S": # -> E
                q.put((pnext,"E"))  
            if m=="W": # -> N
                q.put((pnext,"N"))
            if m=="N": # -> W                
                q.put((pnext,"W"))
        
    return energized

def part1(infile):
    grid = readInput16(infile)
    energized = propagate_beam(grid)
    return len(energized.keys())

In [29]:
print("Test 1:",part1("input.txt"))

Test 1: 7543


In [30]:
def part2(infile):
    grid = readInput16(infile)
    
    starts  = [ ((x,-1),"S") for x in range(len(grid[0])) ] # top
    starts += [ ((-1,y),"E") for y in range(len(grid))    ] # left
    starts += [ ((x,len(grid)),"N") for x in range(len(grid[0])) ] # bottom
    starts += [ ((len(grid[0]),y),"E") for y in range(len(grid)) ] # left
    
    energ = []
    for s in starts:
        energized = propagate_beam(grid,s)
        energ.append(len(energized.keys()))
    
    return max(energ)

In [31]:
print("Test 1:",part2("input.txt"))

Test 1: 8231
